In [9]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.datasets import make_multilabel_classification
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import xgboost as xgb
import joblib
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import datetime
import warnings
warnings.filterwarnings("ignore") 
plt.rc('font', family='NanumGothic') 
from sklearn.preprocessing import RobustScaler
#print (plt.rcParams['font.family'] )

In [128]:
X_test.shape

(7, 21)

In [10]:
df_B = pd.read_csv('시간별_발생건수_B.csv')
df_C = pd.read_csv('시간별_발생건수_C.csv')
#b,c지역 합치기
frame = [df_B,df_C]
data=pd.concat(frame)
data
#발생건수 전처리
def change(values):
    if values >= 1 :
        return 1
    else:
        return 0
data['발생건수']= data['발생건수'].apply(change)
data.drop(['월','일','시','Unnamed: 0'], axis= 1, inplace= True) 


X_features = data.iloc[:, :-1] #특성세트와
y_labels = data.iloc[:,-1]     #target label 분리
#print(X_features.info())
X_features['일시']= pd.to_datetime(X_features['일시'])
X_features['DATE'] = X_features['일시'].dt.strftime('%Y-%m-%d')

X_features = X_features.drop('일시', axis=1)
user = input('ex. 20211205 *12월25일 이전만 입력 : ')

year = user[:4]
month = user[4:6]
days = user[6:]

date = year + '-' + month + '-' + days
date = pd.date_range(user, periods=7, freq='d')
date = [i.strftime('%Y-%m-%d') for i in date]


pred = X_features.loc[X_features['DATE'].isin(date)] #예측에 사용할 데이터 셋 빼기
#pred =pred.drop(['DATE'],axis=1) #예측에 사용할 데이터 셋에 대한 처리

X_features = X_features.loc[~X_features['DATE'].isin(X_features)]
X_features = X_features.reset_index(drop=True)
X_features =X_features.drop(['DATE'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=0,shuffle=True) #데이터 분리


str_col = ['지역','방향'] #지역과 방향 라벨 인코딩 하기
for i in str_col:
    le = LabelEncoder()
    le=le.fit(X_train[i])
    X_train[i]=le.transform(X_train[i])
    for label in np.unique(X_test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    X_test[i]=le.transform(X_test[i])
    pred[i] = le.transform(pred[i])

scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
sm = SMOTE(random_state=0, sampling_strategy=0.6)
smote = SMOTE(random_state=0)
X_train_over, y_train_over = smote.fit_resample(X_train, y_train) 

#랜덤포레스트 모델 학습
rfc = RandomForestClassifier(random_state=0,n_estimators=100)
rfc.fit(X_train_over, y_train_over)
y_pred = rfc.predict(X_test)
y_prob= rfc.predict_proba(X_test)[:,1] >0.1
print('Model의 Accuracy : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))



ex. 20211205 *12월25일 이전만 입력 :  20210703


Model의 Accuracy : 0.9849
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3470
           1       0.29      0.26      0.27        38

    accuracy                           0.98      3508
   macro avg       0.64      0.63      0.63      3508
weighted avg       0.98      0.98      0.98      3508



In [10]:
from sklearn.metrics import confusion_matrix

In [12]:
X_test

array([[ 0.        ,  0.39285714,  0.69817073, ...,  0.53925926,
         0.        ,  0.5       ],
       [ 0.        ,  0.125     , -0.58536585, ...,  0.28444444,
         0.        ,  0.25      ],
       [ 0.        , -0.25595238, -0.40853659, ..., -0.60444444,
         0.        ,  0.5       ],
       ...,
       [ 0.        ,  0.14285714, -1.17682927, ..., -0.46814815,
         0.        , -0.25      ],
       [ 1.        ,  0.42857143,  0.38414634, ...,  0.48      ,
         0.        , -0.75      ],
       [ 0.        , -0.86309524, -2.00914634, ...,  0.7762963 ,
         0.        ,  0.5       ]])

In [13]:
print(classification_report(y_test, y_prob))

              precision    recall  f1-score   support

           0       0.99      0.93      0.96      3470
           1       0.08      0.50      0.13        38

    accuracy                           0.93      3508
   macro avg       0.54      0.72      0.55      3508
weighted avg       0.98      0.93      0.95      3508



In [14]:
y_prob= rfc.predict_proba(X_test)[:,1] >0.1

In [3]:
X_test[:,1]

array([ 0.39285714,  0.125     , -0.25595238, ...,  0.14285714,
        0.42857143, -0.86309524])

In [4]:
confusion_matrix(y_test, y_pred)

array([[3445,   25],
       [  28,   10]])

In [1]:
confusion_matrix(y_test, y_prob)

NameError: name 'confusion_matrix' is not defined

In [11]:
day = pred['DATE']
day = pd.DataFrame(day)
del pred['DATE']
y_pred = rfc.predict(pred)
len(y_pred)
df = pd.DataFrame(y_pred)
pred = pred.reset_index()
day = day.reset_index()

RESULT = pd.concat([pred,df,day],axis=1)
#['남동풍' '남서풍' '남풍' '동풍' '북동풍' '북서풍' '북풍' '서풍']
RESULT['지역'] = ['B' if i==0 else 'C' for i in RESULT['지역']]
#RESULT['방향'].replace([0,1,2,3,4,5,6,7],['남동풍' '남서풍' '남풍' '동풍' '북동풍' '북서풍' '북풍' '서풍'])
def go(values):
    if values ==0:
        return '남동풍'
    elif values ==1:
        return '남서풍'
    elif values ==2:
        return '남풍'
    elif values ==3:
        return '동풍'
    elif values ==4:
        return '북동풍'
    elif values ==5:
        return '북서풍'
    elif values ==6:
        return '북풍'
    else:
        return '서풍'
RESULT['방향'] = RESULT['방향'].apply(go)
RESULT[0] = ['O' if i>0 else 'X' for i in RESULT[0]]
RESULT=RESULT.drop(['index'],axis=1)
RESULT.rename(columns={0:'민원발생여부'})

,지역,기온(°C),습도(%),풍속(m/s),풍향(360°),일사량(w/㎡),PM10(㎍/㎥),PM2.5(㎍/㎥),오존(ppm),증기압(hPa),현지기압(hPa),지면온도(°C),30cm 지중온도(°C),강수량(mm),방향,민원발생여부,DATE
0,B,26.5,62.5,0.8,162.1,1.1,14.0,3.0,0.030,24.4,1002.7,24.2,27.2,0.0,남풍,X,2021-07-03
1,B,25.6,65.9,1.1,176.7,1.1,11.0,8.0,0.035,23.7,1002.4,23.3,27.1,0.0,남풍,X,2021-07-03
2,B,24.9,68.0,1.0,175.3,1.1,9.0,3.0,0.035,23.8,1001.9,22.8,27.0,0.0,남풍,X,2021-07-03
3,B,24.4,69.6,1.0,172.9,1.1,12.0,4.0,0.033,23.4,1001.6,22.3,26.9,0.0,남풍,X,2021-07-03
4,B,24.1,69.9,0.9,169.8,1.1,14.0,5.0,0.033,22.9,1001.4,21.9,26.8,0.0,남풍,X,2021-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,C,28.9,58.6,1.6,283.0,107.0,40.0,18.0,0.047,26.8,997.4,26.8,27.0,0.0,북서풍,X,2021-07-09
333,C,27.5,63.3,1.7,275.0,14.0,24.0,14.0,0.042,26.0,997.9,24.8,27.1,0.0,북서풍,X,2021-07-09
334,C,26.6,65.8,1.0,275.0,0.0,25.0,13.0,0.037,25.8,998.9,24.2,27.2,0.0,북서풍,X,2021-07-09
335,C,25.9,68.1,0.3,204.0,0.0,27.0,14.0,0.037,26.0,999.2,23.6,27.2,0.0,남서풍,X,2021-07-09


In [110]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score

In [111]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [112]:
print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score with 10 decision-trees : 0.9909


In [113]:
model = RandomForestClassifier(n_estimators=100, random_state=0)
# fit the model to the training set
#rfc_100.fit(X_train, y_train)

In [114]:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
#y_pred_100 = rfc_100.predict(X_test)
#print('Model accuracy score with 100 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred_100)))

RandomForestClassifier(random_state=0)

In [115]:
class RandomForestClassifier:
    def __init__(self):
        self.model = RandomForestClassifier()
        self.feature_column_names = list()

    def train(self, X_train, y_train):
        # self.feature_column_names records the column names used for training.
        # It is recommended to set this attribute before training so that the
        # feature columns used in predict and train methods have the same names.
        self.feature_column_names = X_train.columns.tolist()
        self.model.fit(X_train, y_train)

    def predict(self, test):
        # The feature columns used for prediction MUST have the same names as the ones for training.
        # The name of score column ("Scored Labels" in this case) MUST be different from any other columns in input data.
        return pd.DataFrame(
            {'Scored Labels': self.model.predict(test[self.feature_column_names]), 
             'probabilities': self.model.predict_proba(test[self.feature_column_names])[:, 1]}
        )

In [116]:
import pandas as pd

# The entry point function MUST have two input arguments:
#   Param<dataframe1>: a pandas.DataFrame
#   Param<dataframe2>: a pandas.DataFrame
def azureml_main(dataframe1 = None, dataframe2 = None):

    from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve
    import pandas as pd
    import numpy as np

    scores = dataframe1.ix[:, ("income", "Scored Labels", "probabilities")]
    ytrue = np.array([0 if val == '<=50K' else 1 for val in scores["income"]])
    ypred = np.array([0 if val == '<=50K' else 1 for val in scores["Scored Labels"]])    
    probabilities = scores["probabilities"]

    accuracy, precision, recall, auc = \
    accuracy_score(ytrue, ypred),\
    precision_score(ytrue, ypred),\
    recall_score(ytrue, ypred),\
    roc_auc_score(ytrue, probabilities)

    metrics = pd.DataFrame();
    metrics["Metric"] = ["Accuracy", "Precision", "Recall", "AUC"];
    metrics["Value"] = [accuracy, precision, recall, auc]

    return metrics,

In [117]:
import pandas as pd

In [118]:
data = pd.read_csv('A측정소_기상자료(1hr)_2021년.csv',encoding='euc-kr')
del data['지역']
data

,일시,기온(°C),습도(%),풍속(m/s),풍향(360°),일사량(w/㎡),PM10(㎍/㎥),PM2.5(㎍/㎥),오존(ppm),증기압(hPa),현지기압(hPa),지면온도(°C),30cm 지중온도(°C),강수량(mm)
0,2021-01-01 1:00,-8.7,89.2,0.1,340.0,0.0,32.0,22.0,0.003,3.0,1018.9,-6.5,3.0,0.0
1,2021-01-01 2:00,-8.9,89.0,0.6,227.3,0.0,34.0,28.0,0.003,3.0,1019.2,-6.8,2.9,0.0
2,2021-01-01 3:00,-9.5,88.4,0.8,231.4,0.0,36.0,30.0,0.003,2.9,1019.5,-7.1,2.9,0.0
3,2021-01-01 4:00,-9.8,87.3,0.6,231.5,0.0,29.0,28.0,0.002,2.8,1019.5,-7.3,2.8,0.0
4,2021-01-01 5:00,-10.0,87.5,1.1,225.8,0.0,32.0,26.0,0.003,2.8,1019.0,-7.6,2.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,2021-12-31 19:00,-4.2,45.8,0.9,6.8,0.0,21.0,5.0,0.034,1.9,1023.4,-3.9,2.3,0.0
8755,2021-12-31 20:00,-4.6,47.2,0.9,348.2,0.0,25.0,7.0,0.033,2.0,1023.7,-4.7,2.3,0.0
8756,2021-12-31 21:00,-5.1,48.9,0.5,29.8,0.0,27.0,8.0,0.033,2.0,1023.5,-5.4,2.3,0.0
8757,2021-12-31 22:00,-6.3,57.1,0.5,238.3,0.0,25.0,8.0,0.016,2.1,1023.9,-6.0,2.3,0.0
